# Mengambil Data Researcher di Directory ITERA

## Langkah 1: Instalasi Library Python

Jika belum menginstal library ini, buka terminal atau command prompt dan jalankan perintah berikut:

```
conda install -c conda-forge requests beautifulsoup4 pandas
```

## Langkah 2: Memahami Struktur Website

Sebelum menulis kode, kita perlu memahami bagaimana data diatur di website `https://directory.itera.ac.id`. Mari kita buka browser dan amati:

* Halaman Utama (`https://directory.itera.ac.id`): Terlihat ada daftar program studi (Prodi). Kita perlu mengambil tautan ke setiap halaman Prodi.
* Halaman Prodi (`https://directory.itera.ac.id/researcher/listed/88`): Di halaman ini, ada daftar nama dosen. Setiap nama dosen adalah tautan ke halaman profil dosennya. Kita perlu mengambil tautan ke setiap profil dosen.
* Halaman Profil Dosen (`https://directory.itera.ac.id/researcher/id/gpKIfI9pZa9ql2JkaH6khJhjZWiXmGhoZLN4gJZjamVplmNjYoWigA%3D%3D`): Di halaman ini, kita akan menemukan nama dosen dan informasi kepakarannya (biasanya di bagian "Bidang Keilmuan" atau sejenisnya).

## Langkah 3: Menulis Kode Python

### Konfigurasi Awal

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import pandas as pd
import time # Untuk memberikan jeda agar tidak membebani server

# --- Konfigurasi Awal ---
# Pastikan base_url bersih dan hanya domain utama
base_url = "https://directory.itera.ac.id" 

# BARIS KODE BARU: Tentukan kode program studi yang ingin Anda tarik datanya di sini.
# Anda bisa menambahkan atau menghapus angka kode unik di dalam list ini.
prodi_codes_to_scrape = [
    9,   # Perencanaan Wilayah dan Kota
    10,  # Teknik Geomatika
    11,  # Teknik Sipil
    39,  # Teknologi Pangan
    88,  # Sains Data
    12   # Teknik Informatika
] 


### Bagian 1: Mengambil Daftar Tautan Program Studi

In [2]:
# --- Bagian 1: Inisialisasi Tautan Program Studi ---
print("Langkah 1: Menginisialisasi tautan untuk daftar peneliti/dosen berdasarkan kode prodi.")
prodi_links = []
for code in prodi_codes_to_scrape:
    # URL untuk daftar peneliti/dosen spesifik
    list_url = f"{base_url}/researcher/listed/{code}"
    prodi_links.append({
        "name": f"Daftar Peneliti/Dosen (Kode: {code})",
        "url": list_url,
        "code": code # Simpan kode prodi untuk pemetaan nanti
    })
    print(f"  Akan memproses: {list_url}")
print(f"  Ditemukan {len(prodi_links)} daftar peneliti/dosen untuk diproses.")

all_lecturer_profiles = []
lecturer_data = []


Langkah 1: Menginisialisasi tautan untuk daftar peneliti/dosen berdasarkan kode prodi.
  Akan memproses: https://directory.itera.ac.id/researcher/listed/9
  Akan memproses: https://directory.itera.ac.id/researcher/listed/10
  Akan memproses: https://directory.itera.ac.id/researcher/listed/11
  Akan memproses: https://directory.itera.ac.id/researcher/listed/39
  Akan memproses: https://directory.itera.ac.id/researcher/listed/88
  Akan memproses: https://directory.itera.ac.id/researcher/listed/12
  Ditemukan 6 daftar peneliti/dosen untuk diproses.


### Bagian 2: Mengambil Tautan Profil Dosen dari Setiap 'Program Studi' (Daftar Peneliti/Dosen)

In [3]:
# --- Bagian 2: Mengambil Tautan Profil Dosen dari Setiap Daftar Peneliti/Dosen ---
print("\nLangkah 2: Mengambil tautan profil dosen dari setiap daftar peneliti/dosen...")
for prodi in prodi_links:
    prodi_name_display = prodi['name'] # Nama prodi untuk display (misal: "Daftar Peneliti/Dosen (Kode: XX)")
    prodi_code = prodi['code'] # Kode numerik prodi
    prodi_url = prodi['url']
    print(f"  Memproses daftar: {prodi_name_display} ({prodi_url})")

    try:
        response = requests.get(prodi_url)
        response.raise_for_status() # Akan memunculkan error jika request gagal (misal: 404, 500)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Temukan semua tautan yang mengarah ke profil dosen.
        # Tautan ini harus memiliki href mengandung '/researcher/id/'
        lecturer_profile_sections = soup.find_all('a', href=lambda href: href and '/researcher/id/' in href and not '/fakultas/' in href)

        for profile_link_tag in lecturer_profile_sections:
            href_raw = profile_link_tag.get('href') # Ambil href mentah
            
            if href_raw:
                # --- PEMBENTUKAN URL YANG KRUSIAL UNTUK MENGHINDARI DUPLIKASI ---
                # Menggunakan urlparse untuk memecah href_raw
                parsed_href = urlparse(href_raw)

                # Jika href_raw sudah memiliki skema (http/https) dan network location (domain),
                # berarti itu sudah URL absolut dan tidak perlu digabungkan dengan base_url.
                if parsed_href.scheme and parsed_href.netloc:
                    full_profile_url = href_raw
                else:
                    # Jika tidak, itu adalah path relatif. Gabungkan dengan base_url.
                    # urljoin akan menangani jika href_raw dimulai dengan '/' atau tidak.
                    full_profile_url = urljoin(base_url, href_raw)
                
                # --- Tambahan Defensive: Sanitasi Akhir untuk URL yang sangat aneh ---
                # Terkadang ada sisa duplikasi, ini mencoba membersihkannya jika masih ada
                if 'directory.itera.ac.idhttps://directory.itera.ac.id' in full_profile_url:
                    full_profile_url = full_profile_url.replace('https://directory.itera.ac.idhttps://directory.itera.ac.id', 'https://directory.itera.ac.id')
                elif 'directory.itera.ac.idhttps' in full_profile_url: # Mengatasi host yang malformasi
                    # Coba perbaiki jika 'https' menempel di domain
                    full_profile_url = full_profile_url.replace('directory.itera.ac.idhttps', 'directory.itera.ac.id')
                    if not full_profile_url.startswith('https://'): # Pastikan skema tetap ada
                        full_profile_url = 'https://' + full_profile_url.lstrip('/')

                # Memastikan tidak ada duplikasi profil dosen yang sama
                # Simpan juga kode prodi asli bersama URL
                if full_profile_url not in [p['url'] for p in all_lecturer_profiles]:
                    all_lecturer_profiles.append({"prodi_code": prodi_code, "url": full_profile_url})

        time.sleep(1) # Jeda 1 detik antar permintaan halaman daftar dosen
    except requests.exceptions.RequestException as e:
        print(f"  ERROR: Gagal mengambil halaman daftar dosen {prodi_url}. Detail: {e}")
    except Exception as e:
        print(f"  ERROR: Terjadi kesalahan umum saat memproses daftar dosen {prodi_url}. Detail: {e}")

print(f"  Total ditemukan {len(all_lecturer_profiles)} tautan profil dosen unik.")



Langkah 2: Mengambil tautan profil dosen dari setiap daftar peneliti/dosen...
  Memproses daftar: Daftar Peneliti/Dosen (Kode: 9) (https://directory.itera.ac.id/researcher/listed/9)
  Memproses daftar: Daftar Peneliti/Dosen (Kode: 10) (https://directory.itera.ac.id/researcher/listed/10)
  Memproses daftar: Daftar Peneliti/Dosen (Kode: 11) (https://directory.itera.ac.id/researcher/listed/11)
  Memproses daftar: Daftar Peneliti/Dosen (Kode: 39) (https://directory.itera.ac.id/researcher/listed/39)
  Memproses daftar: Daftar Peneliti/Dosen (Kode: 88) (https://directory.itera.ac.id/researcher/listed/88)
  Memproses daftar: Daftar Peneliti/Dosen (Kode: 12) (https://directory.itera.ac.id/researcher/listed/12)
  Total ditemukan 166 tautan profil dosen unik.


### Bagian 3: Mengambil Nama Dosen dan Kepakaran dari Setiap Profil Dosen

In [5]:
print("\nLangkah 3: Mengambil data nama dosen dan kepakaran dari setiap profil dosen...")
for i, profile in enumerate(all_lecturer_profiles):
    profile_url = profile['url']
    prodi_code = profile['prodi_code'] # Ambil kode prodi dari data yang disimpan

    # --- FEEDBACK PROSES: Menampilkan URL yang sedang diproses ---
    print(f"  [{i+1}/{len(all_lecturer_profiles)}] Memproses profil: {profile_url}")
    # --- AKHIR FEEDBACK ---

    try:
        response = requests.get(profile_url)
        response.raise_for_status() # Akan memunculkan error jika request gagal
        soup = BeautifulSoup(response.text, 'html.parser')

        # Ekstrak Nama Dosen
        # Nama dosen berada di tag h3 di dalam <div class="card-body">
        name_tag = None
        card_body_div = soup.find('div', class_='card-body')
        if card_body_div:
            name_tag = card_body_div.find('h3') # Cari h3 di dalam div ini
        
        if not name_tag: # Fallback jika tidak ditemukan dengan metode di atas
            # Jika tidak ditemukan di dalam card-body, coba cari h3 secara umum
            name_tag = soup.find('h3') 

        lecturer_name = name_tag.get_text(strip=True) if name_tag else "<tidak ada nama>"

        # Ekstrak Kepakaran (Bidang Keilmuan)
        # Teks kepakaran berada pada elemen <p> di dalam <h2 class="section-title">Specialities</h2>
        # sekarang diganti menjadi 'Speciality Keywords'
        expertise_area = "<daftar kepakaran belum ditambahkan>"
        # Specialities diganti menjadi 'Speciality Keywords'
        # specialities_heading = soup.find('h2', class_='section-title', string=lambda text: text and 'Specialities' in text)
        specialities_heading = soup.find('h2', class_='section-title', string=lambda text: text and 'Speciality Keywords' in text)
        if specialities_heading:
            expertise_p_tag = specialities_heading.find_next_sibling('p')
            if expertise_p_tag:
                expertise_area = expertise_p_tag.get_text(strip=True)

        lecturer_data.append({
            "Nama Dosen": lecturer_name,
            "Prodi_Code": prodi_code, # Simpan kode prodi di sini
            "Kepakaran": expertise_area,
            "URL Profil": profile_url
        })
        # --- FEEDBACK PROSES: Menampilkan keberhasilan ekstraksi ---
        print(f"    BERHASIL! Data untuk '{lecturer_name}' ditemukan. Kepakaran: '{expertise_area[:70]}...'")
        # --- AKHIR FEEDBACK ---
        time.sleep(1) # Jeda 1 detik antar permintaan profil dosen
    except requests.exceptions.RequestException as e:
        print(f"  ERROR: Gagal mengambil profil {profile_url}. Status HTTP: {e.response.status_code if e.response else 'N/A'}. Detail: {e}")
    except Exception as e:
        print(f"  ERROR: Terjadi kesalahan saat memproses profil {profile_url}. Detail: {e}")

print("\nSelesai mengumpulkan data.")



Langkah 3: Mengambil data nama dosen dan kepakaran dari setiap profil dosen...
  [1/166] Memproses profil: https://directory.itera.ac.id/researcher/id/gpKIfI9paa9nlmFllYFyhY9plphlbmhkaoV3fpBpmJVna5dkk4OisA%3D%3D
    BERHASIL! Data untuk 'M. Bobby Rahman, S.T., M.Si.(Han), Ph.D.' ditemukan. Kepakaran: '<daftar kepakaran belum ditambahkan>...'
  [2/166] Memproses profil: https://directory.itera.ac.id/researcher/id/gpKIfJBiZK9ommFkk4GkhZiSk2NrmWVpZbJ3fZeWY2mabWaUa4Z5hA%3D%3D
    BERHASIL! Data untuk 'Asirin, S.T., M.T.' ditemukan. Kepakaran: '<daftar kepakaran belum ditambahkan>...'
  [3/166] Memproses profil: https://directory.itera.ac.id/researcher/id/gpKIfJBla69oa2Rlk4RxfphmY2tnl2RmaX14hZKWZGVkaGNoYn2jfg%3D%3D
    BERHASIL! Data untuk 'Helmia Adita Fitra, S.T., M.T.' ditemukan. Kepakaran: '<daftar kepakaran belum ditambahkan>...'
  [4/166] Memproses profil: https://directory.itera.ac.id/researcher/id/gpKIfJBmYq%2BWmZNkaISlf8Ril2SWbZSWYrB1rcCXlJeabZSXZrOjfQ%3D%3D
    BERHASIL! Data unt

### Bagian 4: Memproses dan Menyimpan Data ke Format Tabel (CSV/Excel)

In [6]:
# --- Bagian 4: Memproses Data ke Format Tabel (Pandas DataFrame) ---
print("\nLangkah 4: Memproses data dan menampilkannya dalam format tabel...")

if lecturer_data:
    # 1. Buat DataFrame dari data yang dikumpulkan
    df = pd.DataFrame(lecturer_data)

    # 2. Definisikan pemetaan kode prodi ke nama lengkap
    prodi_name_mapping = {
        9: "Perencanaan Wilayah dan Kota",
        10: "Teknik Geomatika",
        11: "Teknik Sipil",
        39: "Teknologi Pangan",
        88: "Sains Data",
        12: "Teknik Informatika",
        36: "Daftar Umum Peneliti/Dosen" # Tambahkan ini jika kode 36 juga digunakan
    }

    # 3. Ubah kolom 'Prodi_Code' menjadi 'Program Studi' dengan nama lengkap
    df['Program Studi'] = df['Prodi_Code'].map(prodi_name_mapping)

    # 4. Hapus kolom 'Prodi_Code' karena sudah diganti dengan 'Program Studi'
    df = df.drop(columns=['Prodi_Code'])

    # 5. Atur ulang urutan kolom agar 'Program Studi' berada di posisi yang logis
    desired_columns_order = ["Nama Dosen", "Program Studi", "Kepakaran", "URL Profil"]
    df = df[desired_columns_order]

    print("\nBerikut adalah 5 baris pertama dari tabel data yang dihasilkan:")
    print(df.head().to_string()) # Menampilkan 5 baris pertama DataFrame ke konsol
    print(f"\nTotal data dosen yang berhasil dikumpulkan: {len(df)} baris.")

    # --- Bagian 5: Mengekspor Data ke CSV ---
    print("\nLangkah 5: Mengekspor data ke file CSV...")
    output_csv_filename = "data_kepakaran_dosen_itera_2050720.csv"
    try:
        df.to_csv(output_csv_filename, index=False, encoding='utf-8', sep = ';')
        print(f"Data berhasil disimpan ke {output_csv_filename} dengan delimiter ';'.")
    except Exception as e:
        print(f"ERROR: Gagal menyimpan data ke CSV. Detail: {e}")

else:
    print("Tidak ada data dosen yang berhasil dikumpulkan untuk diproses menjadi tabel atau diekspor.")


Langkah 4: Memproses data dan menampilkannya dalam format tabel...

Berikut adalah 5 baris pertama dari tabel data yang dihasilkan:
                                         Nama Dosen                 Program Studi                  Kepakaran                                                                                                  URL Profil
0          M. Bobby Rahman, S.T., M.Si.(Han), Ph.D.  Perencanaan Wilayah dan Kota  Kepakaran Tidak Ditemukan    https://directory.itera.ac.id/researcher/id/gpKIfI9paa9nlmFllYFyhY9plphlbmhkaoV3fpBpmJVna5dkk4OisA%3D%3D
1                                Asirin, S.T., M.T.  Perencanaan Wilayah dan Kota  Kepakaran Tidak Ditemukan    https://directory.itera.ac.id/researcher/id/gpKIfJBiZK9ommFkk4GkhZiSk2NrmWVpZbJ3fZeWY2mabWaUa4Z5hA%3D%3D
2                    Helmia Adita Fitra, S.T., M.T.  Perencanaan Wilayah dan Kota  Kepakaran Tidak Ditemukan    https://directory.itera.ac.id/researcher/id/gpKIfJBla69oa2Rlk4RxfphmY2tnl2RmaX14hZKWZGVkaGNoYn2jfg%3D%3D